Soft attention

In [269]:
x = torch.randn(B, T, C)
x

tensor([[[-2.0283e-01, -2.9636e-02],
         [-1.2707e+00, -1.2419e+00],
         [-5.3988e-01, -9.2655e-01],
         [-8.1259e-01,  2.1281e-01],
         [ 5.7511e-01, -7.7577e-02],
         [ 2.8254e-01,  2.4614e+00],
         [-1.1356e+00, -1.6250e+00],
         [ 2.8339e-01, -4.6683e-01]],

        [[-1.9809e-01,  1.3813e+00],
         [-1.4434e+00, -5.6835e-01],
         [ 1.7345e+00,  1.6113e+00],
         [-1.3448e+00,  4.0443e-01],
         [ 6.6593e-01, -1.8187e+00],
         [ 8.6277e-01, -1.7888e+00],
         [ 1.9954e+00,  1.3009e+00],
         [ 1.8678e+00,  6.0636e-01]],

        [[-2.4219e+00, -1.7908e+00],
         [ 5.5150e-01,  9.5523e-02],
         [-6.9393e-01, -6.5913e-01],
         [ 1.1158e+00,  5.8596e-02],
         [ 7.5807e-01, -2.6251e+00],
         [-8.7780e-01,  3.5585e-01],
         [ 3.2531e-01,  4.6225e-01],
         [-3.5128e-01, -1.0840e-01]],

        [[-6.8826e-01,  3.0371e-01],
         [ 2.1860e+00,  2.0890e+00],
         [ 1.2049e+00,  1.2339e+

In [256]:
B, T, C = 4, 8, 2
wei = torch.tril(torch.ones(T, T))
wei

tensor([[1., 0., 0., 0., 0., 0., 0., 0.],
        [1., 1., 0., 0., 0., 0., 0., 0.],
        [1., 1., 1., 0., 0., 0., 0., 0.],
        [1., 1., 1., 1., 0., 0., 0., 0.],
        [1., 1., 1., 1., 1., 0., 0., 0.],
        [1., 1., 1., 1., 1., 1., 0., 0.],
        [1., 1., 1., 1., 1., 1., 1., 0.],
        [1., 1., 1., 1., 1., 1., 1., 1.]])

In [265]:
S = wei.sum(1, keepdim=True)
S

tensor([[1.],
        [2.],
        [3.],
        [4.],
        [5.],
        [6.],
        [7.],
        [8.]])

In [267]:
wei = wei / S
wei

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2500, 0.2500, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.1111, 0.1111, 0.1111, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0625, 0.0625, 0.0625, 0.0625, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0400, 0.0400, 0.0400, 0.0400, 0.0400, 0.0000, 0.0000, 0.0000],
        [0.0278, 0.0278, 0.0278, 0.0278, 0.0278, 0.0278, 0.0000, 0.0000],
        [0.0204, 0.0204, 0.0204, 0.0204, 0.0204, 0.0204, 0.0204, 0.0000],
        [0.0156, 0.0156, 0.0156, 0.0156, 0.0156, 0.0156, 0.0156, 0.0156]])

In [ ]:
xbow = wei @ 